In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

データの読み込み

In [ ]:
df_sub = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
df_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

データの中身を一部確認

In [ ]:
df_train.head()

データの情報をdf.info()で確認できる

In [ ]:
df_train.info()

isnull()：データが欠損値（空の場合）True判定<br>
sum()：Trueの数を数える

In [ ]:
df_train.isnull().sum().head(100)

df.describe()：各列の特徴がわかる，count（データの数）, mean（平均）, std（標準偏差）など

In [ ]:
df_train.describe()

In [ ]:
missing = df_train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

In [ ]:
df_train.isnull().sum()

GrLivAreaをx軸、SalePriceをy軸としてプロット<br>
正の相関が見える、外れ値も確認できる<br>
ちなみにGrLivAreaとはリビングの広さらしい、何故リビングの広さに着目したのか<br>
直観か？

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = df_train["GrLivArea"], y = df_train["SalePrice"])
plt.ylabel("SalePrice", fontsize=13)
plt.xlabel("GrLivArea", fontsize=13)
plt.show()

seabornのdistplotメソッドでヒストグラムを作成<br>
引数はデータフレーム型

In [ ]:
ax = sns.distplot(df_train["SalePrice"])

In [ ]:
ax = sns.boxplot(x = df_train["SalePrice"])

外れ値の除去<br>
リビングが広すぎるおよび家賃が30万未満のデータは外れ値であると判定

In [ ]:
df_train = df_train.drop(df_train[(df_train["GrLivArea"]>4000) & (df_train["SalePrice"]<300000)].index)

外れ値が消えたことを確認

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_train["GrLivArea"], df_train["SalePrice"])
plt.ylabel("SalePrice", fontsize=13)
plt.xlabel("GrLivArea", fontsize=13)
plt.show()

In [ ]:
ax = sns.boxplot(x=df_train["SalePrice"])

In [ ]:
ax = sns.distplot(df_train["SalePrice"])

ntrain：訓練データの形状

In [ ]:
ntrain = df_train.shape
display(ntrain)

各列において欠損値の割合を求める<br>
各列において欠損値の割合が0のカラムは削除

In [ ]:
df_train_nu = (df_train.isnull().sum()/len(df_train)) * 100
df_train_nu = df_train_nu.drop(df_train_nu[df_train_nu == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({"Missing Ratio" : df_train_nu})
missing_data.head(100)

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation="90")
sns.barplot(x=df_train_nu.index, y=df_train_nu)
plt.xlabel("Features", fontsize=15)
plt.ylabel("Percent of missing values", fontsize=15)
plt.title("Percent missing data by feature", fontsize=15)

In [ ]:
df_train.info()

In [ ]:
df_train["PoolQC"].isnull().sum()

In [ ]:
df_train["MiscFeature"].isnull().sum()

In [ ]:
df_train["Fence"].isnull().sum()

In [ ]:
df_train["FireplaceQu"].isnull().sum()

In [ ]:
df_train.drop(["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], axis=1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train.isnull().sum()

2回目<br>
各列でデータの欠損率を算出

In [ ]:
df_train_nu = (df_train.isnull().sum() / len(df_train)) * 100
df_train_nu = df_train_nu.drop(df_train_nu[df_train_nu==0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({"Missing Ratio" : df_train_nu})
missing_data.head(30)

In [ ]:
df_train.LotFrontage

LotFrontage：物件に面している通りの直線距離<br>
グラフの形がSalePriceと似ている

In [ ]:
ax = sns.distplot(df_train["LotFrontage"])

In [ ]:
ax = sns.boxplot(x=df_train["LotFrontage"])

LotFrontageの欠損値をデータの中央値で埋める

In [ ]:
df_train["LotFrontage"].fillna(df_train.LotFrontage.median(), inplace=True)

In [ ]:
ax = sns.distplot(df_train["LotFrontage"])

In [ ]:
ax = sns.boxplot(x=df_train["LotFrontage"])

Garage：車庫もしくは作業場

In [ ]:
df_train.GarageType

In [ ]:
df_train["GarageType"].isnull().sum()

In [ ]:
df_train["GarageType"].fillna(df_train["GarageType"].mode()[0], inplace=True)

In [ ]:
df_train["GarageType"].isnull().sum()

GarageYrBlt：GarageYearBuilt、ガレージの建築年

In [ ]:
df_train.GarageYrBlt

In [ ]:
ax = sns.distplot(df_train["GarageYrBlt"])

In [ ]:
df_train["GarageYrBlt"].isnull().sum()

In [ ]:
df_train.GarageYrBlt.median()

In [ ]:
df_train["GarageYrBlt"].fillna(df_train.GarageYrBlt.median(), inplace=True)

In [ ]:
df_train["GarageYrBlt"].isnull().sum()

In [ ]:
df_train.GarageFinish

In [ ]:
df_train["GarageFinish"].fillna(df_train.GarageFinish.mode()[0], inplace=True)

In [ ]:
df_train["GarageFinish"].isnull().sum()

In [ ]:
df_train["GarageQual"]

In [ ]:
df_train["GarageQual"].fillna(df_train.GarageQual.mode()[0], inplace=True)

In [ ]:
df_train["GarageQual"].isnull().sum()

In [ ]:
df_train.GarageCond

In [ ]:
df_train["GarageCond"].fillna(df_train.GarageCond.mode()[0], inplace=True)

In [ ]:
df_train["GarageCond"].isnull().sum()

In [ ]:
df_train.BsmtExposure

In [ ]:
df_train["BsmtExposure"].fillna(df_train.BsmtExposure.mode()[0], inplace=True)

In [ ]:
df_train["BsmtExposure"].isnull().sum()

In [ ]:
df_train.BsmtFinType2

In [ ]:
df_train["BsmtFinType2"].isnull().sum()

In [ ]:
df_train["BsmtFinType2"].fillna(df_train.BsmtFinType2.mode()[0], inplace=True)

In [ ]:
df_train["BsmtFinType2"].isnull().sum()

In [ ]:
df_train.BsmtQual

In [ ]:
df_train["BsmtQual"].isnull().sum()

In [ ]:
df_train["BsmtQual"].fillna(df_train.BsmtQual.mode()[0], inplace=True)

In [ ]:
df_train["BsmtQual"].isnull().sum()

In [ ]:
df_train.BsmtCond

In [ ]:
df_train["BsmtCond"].isnull().sum()

In [ ]:
df_train["BsmtCond"].fillna(df_train.BsmtCond.mode()[0], inplace=True)

In [ ]:
df_train["BsmtCond"].isnull().sum()

In [ ]:
df_train.BsmtFinType1

In [ ]:
df_train["BsmtFinType1"].isnull().sum()

In [ ]:
df_train["BsmtFinType1"].fillna(df_train.BsmtFinType1.mode()[0], inplace=True)

In [ ]:
df_train["BsmtFinType1"].isnull().sum()

In [ ]:
df_train.MasVnrType

In [ ]:
df_train.MasVnrArea

In [ ]:
df_train["MasVnrArea"].isnull().sum()

In [ ]:
ax = sns.distplot(df_train["MasVnrArea"])

In [ ]:
df_train["MasVnrArea"].fillna(df_train.MasVnrArea.median(), inplace=True)

In [ ]:
df_train["MasVnrArea"].isnull().sum()

In [ ]:
df_train.MasVnrType

In [ ]:
df_train.Electrical

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train["Electrical"].fillna(df_train.Electrical.mode()[0], inplace=True)
df_train["MasVnrType"].fillna(df_train.MasVnrType.mode()[0], inplace=True)

In [ ]:
df_train_nu = (df_train.isnull().sum() / len(df_train)) * 100
df_train_nu = df_train_nu.drop(df_train_nu[df_train_nu==0].index).sort_values(ascending=False)[:30]

In [ ]:
missing_data = pd.DataFrame({"Missin Ratio" : df_train_nu})
missing_data.head()

In [ ]:
corrmat = df_train.corr()
plt.subplots(figsize=(20, 20))
sns.heatmap(corrmat, vmax=1.1, square=True)

In [ ]:
df_train.info()

In [ ]:
df_test['LotFrontage'].fillna(df_test.LotFrontage.median(), inplace = True)
df_test['MasVnrArea'].fillna(df_test.MasVnrArea.median(), inplace = True)
df_test['TotalBsmtSF'].fillna(df_test.TotalBsmtSF.mode()[0], inplace = True)
df_test['BsmtFinSF1'].fillna(df_test.BsmtFinSF1.median(), inplace = True)
df_test['BsmtFinSF2'].fillna(df_test.BsmtFinSF2.mode()[0], inplace = True)
df_test['BsmtUnfSF'].fillna(df_test.BsmtUnfSF.median(), inplace = True)
df_test['GarageYrBlt'].fillna(df_test.GarageYrBlt.median(), inplace = True)
df_test['BsmtFullBath'].fillna(df_test.BsmtFullBath.mode()[0], inplace = True)
df_test['BsmtHalfBath'].fillna(df_test.BsmtHalfBath.mode()[0], inplace = True)
df_test['GarageCars'].fillna(df_test.GarageCars.mode()[0], inplace = True)
df_test['GarageArea'].fillna(df_test.GarageArea.mean(), inplace = True)

In [ ]:
x_data = df_train.drop("SalePrice", axis=1)
y_data = df_train["SalePrice"]

Scaling

In [ ]:
df_train.select_dtypes(include=["object"])

In [ ]:
df_train_object = df_train.select_dtypes(include=["object"])

In [ ]:
df_train_object.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
object_cols = ('MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition')

LabelEncoder：文字にラベルをつける(0~N)、同じ文字＝同じID

In [ ]:
for i in object_cols:
    lb_encoder = LabelEncoder()
    lb_encoder.fit(list(df_train[i].values))
    df_train[i] = lb_encoder.transform(list(df_train[i].values))

In [ ]:
df_train.select_dtypes(include=["object"])

In [ ]:
df_train.info()

Dealing with Test Set

In [ ]:
df_test["BsmtFullBath"]

In [ ]:
df_test["BsmtHalfBath"]

In [ ]:
display(df_test["BsmtFinSF1"])
ax = sns.distplot(df_test["BsmtFinSF1"])

In [ ]:
df_test["GarageCars"]

In [ ]:
display(df_test["TotalBsmtSF"])
ax = sns.distplot(df_test["TotalBsmtSF"])

In [ ]:
df_test["BsmtFinSF2"]

In [ ]:
df_test["BsmtUnfSF"]

In [ ]:
df_test_nu = (df_test.isnull().sum()/len(df_test)) * 100
df_test_nu = df_test_nu.drop(df_train_nu[df_test_nu==0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({"Missing Ratio" : df_test_nu})
missing_data.head(100)

In [ ]:
df_test.drop(["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], axis=1, inplace=True)

In [ ]:
df_test['LotFrontage'].fillna(df_test.LotFrontage.median(), inplace = True)
df_test['GarageType'].fillna(df_test['GarageType'].mode()[0], inplace = True)
df_test['GarageYrBlt'].fillna(df_test.GarageYrBlt.median(), inplace = True)
df_test['GarageFinish'].fillna(df_test.GarageFinish.mode()[0], inplace = True)
df_test['GarageQual'].fillna(df_test.GarageQual.mode()[0], inplace = True)
df_test['GarageCond'].fillna(df_test.GarageCond.mode()[0], inplace = True)
df_test['BsmtExposure'].fillna(df_test.BsmtExposure.mode()[0], inplace = True)
df_test['BsmtFinType2'].fillna(df_test.BsmtFinType2.mode()[0], inplace = True)
df_test['BsmtQual'].fillna(df_test.BsmtQual.mode()[0], inplace = True)
df_test['BsmtCond'].fillna(df_test.BsmtCond.mode()[0], inplace = True)
df_test['BsmtFinType1'].fillna(df_test.BsmtFinType1.mode()[0], inplace = True)
df_test['MasVnrArea'].fillna(df_test.MasVnrArea.median(), inplace = True)
df_test['Electrical'].fillna(df_test.Electrical.mode()[0], inplace = True)
df_test['MasVnrType'].fillna(df_test.MasVnrType.mode()[0], inplace = True)
df_test['BsmtFullBath'].fillna(df_test.BsmtFullBath.mode()[0], inplace = True)
df_test['BsmtHalfBath'].fillna(df_test.BsmtHalfBath.mode()[0], inplace = True)
df_test['BsmtFinSF1'].fillna(df_test.BsmtFinSF1.median(), inplace = True)
df_test['GarageCars'].fillna(df_test.GarageCars.mode()[0], inplace = True)
df_test['GarageArea'].fillna(df_test.GarageArea.mean(), inplace = True)
df_test['TotalBsmtSF'].fillna(df_test.TotalBsmtSF.mode()[0], inplace = True)
df_test['BsmtFinSF2'].fillna(df_test.BsmtFinSF2.mode()[0], inplace = True)
df_test['BsmtUnfSF'].fillna(df_test.BsmtUnfSF.median(), inplace = True)
df_test['MSZoning'].fillna(df_test.MSZoning.mode()[0], inplace = True)
df_test['Functional'].fillna(df_test.Functional.mode()[0], inplace = True)
df_test['Utilities'].fillna(df_test.Utilities.mode()[0], inplace = True)
df_test['Exterior2nd'].fillna(df_test.Exterior2nd.mode()[0], inplace = True)
df_test['Exterior1st'].fillna(df_test.Exterior1st.mode()[0], inplace = True)
df_test['SaleType'].fillna(df_test.SaleType.mode()[0], inplace = True)
df_test['KitchenQual'].fillna(df_test.KitchenQual.mode()[0], inplace = True)

In [ ]:
df_test_nu = (df_test.isnull().sum() / len(df_test)) * 100
df_test_nu = df_test_nu.drop(df_train_nu[df_test_nu == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :df_test_nu})
missing_data.head(100)

In [ ]:
df_test.select_dtypes(include=["object"])
df_test_object = df_test.select_dtypes(include=["object"])
df_test_object.columns

In [ ]:
for i in object_cols:
    lb_encoder = LabelEncoder()
    lb_encoder.fit(list(df_test[i].values))
    df_test[i] = lb_encoder.transform(list(df_test[i].values))

In [ ]:
df_test.select_dtypes(include=["object"])
df_test.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
x_data = df_train.drop("SalePrice", axis=1)
y_data = df_train["SalePrice"]

In [ ]:
df_train.info()

In [ ]:
x_data.info()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)
x_train.info()

In [ ]:
x_data

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)
predictions = model.predict(x_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score

In [ ]:
print("Mean Absolute Error(MAE) : ", mean_absolute_error(y_test, predictions))
print("Mean Squared Error(MSE) : ", mean_squared_error(y_test, predictions))
print("Root Mean Squared Error(RMSE) : ", np.sqrt(mean_squared_error(y_test, predictions)))
print("Explained Variance Score(EVS) : ", explained_variance_score(y_test, predictions))
print("R2 : ", r2_score(y_test, predictions))

In [ ]:
print("R2_rounded:", (r2_score(y_test, predictions)).round(2))
r2 = r2_score(y_test, predictions)
r2_rounded = r2_score(y_test, predictions).round(2)

In [ ]:
from sklearn.linear_model import Ridge

model_ridge = Ridge()
model_ridge.fit(x_train, y_train)
ridge_predictions = model_ridge.predict(x_test)

In [ ]:
print("Mean Absolute Error(MAE):", mean_absolute_error(y_test, ridge_predictions))
print("Mean Squared Error(MSE):", mean_squared_error(y_test, ridge_predictions))
print("Root Mean Squared Error(RMSE):", np.sqrt(mean_squared_error(y_test, ridge_predictions)))
print("Explained Variance Score(EVS:", explained_variance_score(y_test, ridge_predictions))
print("R2:", r2_score(y_test, ridge_predictions))
print("R2 rounded:", r2_score(y_test, ridge_predictions).round(2))
r2 = r2_score(y_test, ridge_predictions)
r2_rounded = r2_score(y_test, ridge_predictions).round(2)